In [1]:
import sqlite3
import re
import calendar # for isleap
import datetime
import pandas as pd

In [377]:
class Book(object):
    def __init__(self, gnc_db):
        self.db = sqlite3.connect(gnc_db)
        self._getCommodities()
        self._createCommodityObjects()
        self._getBookRootAccount()
        self._getBookAccounts()
        self._createAccountObjects()
    
    def _getCommodities(self):
        self.commDF = pd.read_sql_query("SELECT * FROM commodities;", self.db)
        
    def _createCommodityObjects(self):
        self.commodities = []
        for i in range(self.commDF.shape[0]):
            comm = self.commDF.iloc[[i]]
            c = Commodity(self, comm)
            self.commodities.append(c)
                
    def _getBookRootAccount(self):
        """Obtain the root account guid of the gnucash book.
        
        Assumptions:
        - There is only one root account in the book
        """
        r = self.db.execute("SELECT root_account_guid FROM books;").fetchone()
        if r is None:
            raise ValueError("No valid root_account found in the gnucash book")
        else:
            self.rootAccountGuid = r[0]
                        
    def _getBookAccounts(self):
        self.accDF = pd.read_sql_query("SELECT * FROM accounts;", self.db)
        
    def _createAccountObjects(self):
        self.root = Account(self, self.accDF.loc[self.accDF.guid == self.rootAccountGuid], parent=None)
        self.accounts = [self.root]
        for child, parentGuid in self._iterAccountTree(self.root.guid):
            acc = Account(self, child, self.getAccountByGuid(parentGuid))
            self.accounts.append(acc)

    def _iterAccountTree(self, parentGuid):
        children = self._findChildrenOf(parentGuid)
        for i in range(children.shape[0]):
            child = children.iloc[[i]]
            yield (child, parentGuid)
            yield from self._iterAccountTree(child.iloc[0].guid)
        
    def _findChildrenOf(self, parentGuid):
        ch = self.accDF.loc[self.accDF.parent_guid == parentGuid]
        return ch
    
    def getAccountByGuid(self, accGuid):
        for a in self.accounts:
            if a.guid == accGuid:
                return a
        return None
    
    def getAccountByName(self, accName, strict=True):
        for a in self.accounts:
            if a.name == accName:
                return a
        return None
    
    def getCommodityByGuid(self, commGuid):
        for c in self.commodities:
            if c.guid == commGuid:
                return c
        return None
    
    def getCommodityByName(self, commName):
        for c in self.commodities:
            if c.mnemonic == commName:
                return c
        return None
    
    def _printAccounts(self, parent, level=0):
        print(level*" " + parent.name)
        for c in parent.children:
            self._printAccounts(c, level+3)
        
class Account(object):
    ROOT = 1
    ASSET = 2
    CASH = 3
    BANK = 4
    LIABILITY = 5
    CREDIT = 6
    TRADING = 7
    
    def __init__(self, book, accDF, parent):
        acc = accDF.iloc[0]
        self.book = book
        self.guid = acc.guid
        self.name = acc['name']
        self.type = self._type(acc.account_type)
        self.parent_guid = acc.parent_guid
        self.parent = parent
        self.description = acc.description
        self.commodity = self.book.getCommodityByGuid(acc.commodity_guid)
        self.hidden = acc.hidden
        self.placeholder = (acc.placeholder == 1)
        self.children = [] # to be completed via addChild
        if parent is not None:
            parent.addChild(self)
        
    def _type(self, accType):
        if accType == 'ROOT': return self.ROOT
        if accType == 'ASSET': return self.ASSET
        if accType == 'CASH': return self.ROOT
        if accType == 'BANK': return self.BANK
        if accType == 'LIABILITY': return self.LIABILITY
        if accType == 'CREDIT': return self.CREDIT
        if accType == 'TRADING': return self.TRADING
        
    def addChild(self, child):
        if not isinstance(child, Account):
            ValueError("Non Account-type passed for child")
        self.children.append(child)
        
class Commodity(object):
    CURRENCY = 1
    def __init__(self, book, commDF):
        comm = commDF.iloc[0]
        self.book = book
        self.guid = comm.guid
        self.name = comm.mnemonic
        self.fullname = comm.fullname
        self.type = self._type(comm.namespace)
        self.iscurrency = comm.namespace == "CURRENCY"
        
    def _type(self, commType):
        if commType == "CURRENCY": return self.CURRENCY

In [378]:
book = Book("money.gnucash.sql.gnucash")

In [379]:
book._printAccounts(book.root)

Root Account
   Assets
      Current Assets
         Cash EUR
         SGS credit account
         FIB Maestro
         FIB VisaElectron
         Epay Microaccount
         Spain Card
         A/R
         Cash CHF
         PostFinance
            Private
               Blocked
               Savings
            Deposito
            Private-Bills
   Liabilities
      Credit Card CFF Visa
      Credit Card FIB MC
      Loan car
      MyOne
      A/P
   Income
      Bonus
      Gifts Received
      Interest Income
         Checking Interest
         Other Interest
         Savings Interest
      Salary
      Second-hand sells
      Gambling
      ECAP
   Expenses
      Adjustment
      Car
         State Fees
         Gas
         Parking
         Repair and Maintenance
         Membership
         Equipment and supplies
         Peage
      Charity
      Clothes & Shoes
      Education
      Groceries
      Hobbies, Sport, Tourism
      Insurance
         Auto Insurance
         Health 

In [381]:
book.getAccountByName('Private').commodity.type == Commodity.CURRENCY

True